In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex7'
site = 'P301'
method = '7'

In [3]:
files = glob.glob('/Volumes/data/RHESSys_out/P301/'+exname+'/*_basin.daily')

In [4]:
len(files)

10000

In [5]:
params = pd.read_pickle('./data/params_%s_smex_method%s.pcl'%(site,method))

In [6]:
files = pd.DataFrame({'basin_daily':files})

In [7]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [8]:
files['idx'] = files.basin_daily.map(getIDX)

In [9]:
params = pd.merge(params,files, on= 'idx')

In [10]:
params.head()

,SMR,TpeakSWE,peakSWE,ablSeas,Tnosnow,ref,date_ONS,DOAS,date_AS,idx,date_peakSWE,MAP,basin_daily
0,0.014403,113.0,0.521211,37.0,150.0,2009,2009-02-27,15,2008-10-15,0,2009-01-21,0.325,/Volumes/data/RHESSys_out/P301/smex7/smex7_0_b...
1,0.014842,177.0,1.328189,90.0,267.0,2009,2009-06-24,15,2008-10-15,1,2009-03-26,0.041,/Volumes/data/RHESSys_out/P301/smex7/smex7_1_b...
2,0.009731,192.0,0.940332,97.0,289.0,2009,2009-07-16,15,2008-10-15,2,2009-04-10,0.041,/Volumes/data/RHESSys_out/P301/smex7/smex7_2_b...
3,0.007164,116.0,0.779654,109.0,225.0,2009,2009-05-13,15,2008-10-15,3,2009-01-24,0.081,/Volumes/data/RHESSys_out/P301/smex7/smex7_3_b...
4,0.004957,151.0,0.593449,120.0,271.0,2009,2009-06-28,15,2008-10-15,4,2009-02-28,0.041,/Volumes/data/RHESSys_out/P301/smex7/smex7_4_b...


In [11]:
c = p.Client()
view = c.load_balanced_view()

In [12]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [22]:
%px dr = pd.date_range('2008-10-1','2009-10-2',freq='D')

In [23]:
dr = pd.date_range('2008-10-1','2009-10-2',freq='D')

In [24]:
len(dr)

367

In [25]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_table(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans

    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [26]:
#res = view.map(processSM,params.basin_daily[0:1000],params.idx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.idx,params.date_peakSWE,params.date_ONS)

In [27]:
prog = pb.ProgressBar(len(params))

In [28]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

alert.send_alert('barnhatb@colorado.edu','Processing P301 %s has finished'%exname,'Your script has finished')

[                  0%                  ]


In [29]:
res.ready()

True

In [30]:
res.progress

10000

In [31]:
full,melt = zip(*res.result())

In [32]:
idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/params_%s_smex_method%s.pcl'%(site,method)) # load parameters dataset

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['idx','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='idx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_long_full.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [33]:
idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/params_%s_smex_method%s.pcl'%(site,method)) # load parameters dataset

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['idx','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='idx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_long_melt.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)